In [1]:
%load_ext autoreload
%autoreload 2

In [35]:
import gc
import sys
import json
import numpy as np
import pandas as pd
import mml_lib.visualization as viz

from IPython.display import display_html, Markdown, display

sys.path.append('/home/jovyan/work/yh09262/cython_home')
sys.path.append('/home/jovyan/work/yh09262/utils_and_queries')

from get_csv_rows import get_csv_rows
from get_baskets import get_baskets
from get_baskets_cdef import get_baskets_cdef

import UPC_utils_July_2019 as uu

In [3]:
!ls -d ~/datasets/d_train*

/home/jovyan/datasets/d_train_20181030_60
/home/jovyan/datasets/d_train_20190210_90
/home/jovyan/datasets/d_train_20190515_14
/home/jovyan/datasets/d_train_20190723_120_0
/home/jovyan/datasets/d_train_20190723_60_0
/home/jovyan/datasets/d_train_20190723_90_0
/home/jovyan/datasets/d_train_20190809_3_0
/home/jovyan/datasets/d_train_ari_namita
/home/jovyan/datasets/d_train_test


In [11]:
#attr_rows_big = get_csv_rows('/home/jovyan/datasets/d_train_20190723_120_0/ur_production_product_attr.csv',numcols=23)
attr_rows_recent = get_csv_rows('/home/jovyan/datasets/d_train_20190809_3_0/ur_production_product_attr.csv',numcols=23)

In [12]:
len(attr_rows_big), len(attr_rows_recent)

(2751103, 2799177)

In [14]:
attr_r = pd.DataFrame(attr_rows_recent[1:])
attr_r.columns = attr_rows_recent[0]
attr_b = pd.DataFrame(attr_rows_big[1:])
attr_b.columns = attr_rows_big[0]
attr_r.head(3)

,product_id,prod_name,today_original_price,today_retail_price,availability_flag,home_category_id,gmm_id,div_id,department_id,prod_age_txt,...,deal,age_category,prodtype,gender,master_product_flag,recommend_pdp,product_long_description,special_size,view_30_day_count,purch_30_day_count
0,99290475,"MARTHA STEWART COLLECTION ""FAUX BOIS"" REVERSIB...",\N,\N,\N,\N,47,81,609,\N,...,regular,\N,COMFORTER,\N,\N,Y,\N,\N,0,0
1,99290474,"MARTHA STEWART COLLECTION ""FAUX BOIS"" REVERSIB...",\N,\N,\N,\N,47,81,609,\N,...,regular,\N,COMFORTER,\N,\N,Y,\N,\N,0,0
2,99290469,"JESSICA SIMPSON ""KREME"" SLINGBACK PUMP",\N,\N,\N,\N,1,16,394,\N,...,regular,\N,UNKNOWN,\N,\N,Y,\N,\N,0,0


In [15]:
del attr_rows_recent
del attr_rows_big
gc.collect()

0

In [16]:
attr_r.columns

Index(['product_id', 'prod_name', 'today_original_price', 'today_retail_price',
       'availability_flag', 'home_category_id', 'gmm_id', 'div_id',
       'department_id', 'prod_age_txt', 'new_arrival', 'brand',
       'today_price_type_id', 'deal', 'age_category', 'prodtype', 'gender',
       'master_product_flag', 'recommend_pdp', 'product_long_description',
       'special_size', 'view_30_day_count', 'purch_30_day_count'],
      dtype='object')

In [17]:
print("homecat ids: ", attr_r['home_category_id'].nunique())
print("gmm ids: ", attr_r['gmm_id'].nunique())
print("div ids: ", attr_r['div_id'].nunique())
print("department ids: ", attr_r['department_id'].nunique())
print("prodtype: ", attr_r['prodtype'].nunique())

homecat ids:  750
gmm ids:  6
div ids:  57
department ids:  709
prodtype:  339


In [18]:
avail_attr = attr_r.query('availability_flag == "Y" & master_product_flag != "Y"')[['product_id', 'prod_name', 
                                                            'home_category_id', 'gmm_id', 'div_id', 
                                                            'department_id', 'brand', 'prodtype',
                                                            'view_30_day_count', 'purch_30_day_count']]
avail_attr['view_30_day_count'] = avail_attr['view_30_day_count'].astype('int32')
avail_attr['purch_30_day_count'] = avail_attr['purch_30_day_count'].astype('int32')
avail_attr = avail_attr.sort_values(by=['purch_30_day_count','view_30_day_count'],ascending=False)
avail_attr.head()

,product_id,prod_name,home_category_id,gmm_id,div_id,department_id,brand,prodtype,view_30_day_count,purch_30_day_count
1382239,4401441,TOMMY HILFIGER ALL AMERICAN II COTTON BATH TOW...,16853,47,81,601,TOMMY HILFIGER,TOWEL,204522,28160
237719,9386971,DIAMOND CLUSTER STUD EARRINGS (1/10 CT. T.W.) ...,10835,1,35,734,MACY'S,EARRINGS,181846,22514
969965,6945072,CLARKS COLLECTION WOMEN'S BRINKLEY SAIL FLIP-F...,17570,1,16,391,CLARKS,SHOE,304418,17595
1137088,5887722,INSTANT POT DUO60 7-IN-1 PROGRAMMABLE PRESSUR...,7554,47,85,691,INSTANT POT,ELECTRICS,366994,14655
2414824,560829,BALI DOUBLE SUPPORT SPA CLOSURE WIRELESS BRA ...,55799,1,22,251,BALI,BRA,218755,12739


In [19]:
avail_pids = avail_attr['product_id'].unique()
print(len(avail_pids), avail_attr.shape)
print(avail_pids[:5])

728960 (728960, 10)
['4401441' '9386971' '6945072' '5887722' '560829']


In [20]:
avail_attr_purch = avail_attr.query('purch_30_day_count > 0 & view_30_day_count > 0')
avail_attr_purch.tail()

,product_id,prod_name,home_category_id,gmm_id,div_id,department_id,brand,prodtype,view_30_day_count,purch_30_day_count
1287608,4824912,"ROD DESYNE EMILIA 5/8"" CURTAIN ROD 84""-120""",198056,47,81,61,ROD DESYNE,WINDOW_HARDWARE,1,1
1506295,2870324,"ROD DESYNE LENORE SINGLE CURTAIN ROD 160-240""",198056,47,81,61,ROD DESYNE,WINDOW_HARDWARE,1,1
1726353,2294898,FOREVER COLLECTIBLES INDIANAPOLIS COLTS BIG LO...,65701,91,67,912,FOREVER COLLECTIBLES,BACKPACK_MESSENGER,1,1
1942320,1613268,TERVIS TUMBLER LOS ANGELES KINGS 24 OZ. COLOSS...,65701,91,67,912,TERVIS TUMBLER,BARWARE,1,1
1942364,1613224,TERVIS TUMBLER TAMPA BAY BUCCANEERS 24 OZ. COL...,65701,91,67,912,TERVIS TUMBLER,BARWARE,1,1


In [28]:
medianpc = int(avail_attr_purch['purch_30_day_count'].median())
meanpc = int(avail_attr_purch['purch_30_day_count'].mean())
std = int(avail_attr_purch['purch_30_day_count'].std())
mm = int((medianpc+meanpc)/2)
mid = int(avail_attr_purch['purch_30_day_count'].max()/2)
print(medianpc, meanpc, mm, mid, std)

7 50 28 14080 214


In [29]:
avail_purch_pids = avail_attr_purch['product_id'].unique()
print(len(avail_purch_pids))
print(avail_purch_pids[:5])

173407
['4401441' '9386971' '6945072' '5887722' '560829']


In [77]:
bestsell_20 = avail_purch_pids[:20]
bestsell_200 = avail_purch_pids[:200]
worstsell_20 = avail_purch_pids[-20:]
worstsell_200 = avail_purch_pids[-200:]
midsell_15good10_20 = np.random.choice(avail_purch_pids[mid-(std*15):mid-(std*10)],size=20,replace=False) # neg = higher rank
midsell_20 = np.random.choice(avail_purch_pids[mid-std:mid+std],size=20,replace=False)
midsell_200 = np.random.choice(avail_purch_pids[mid-std-std:mid+std+std],size=200,replace=False)
midsell_10bad15_20 = np.random.choice(avail_purch_pids[mid+(std*10):mid+(std*15)],size=20,replace=False) # pos = lower rank
sample_20 = np.random.choice(avail_pids,size=20,replace=False)
sample_200 = np.random.choice(avail_pids,size=200,replace=False)
samplep_20 = np.random.choice(avail_purch_pids,size=20,replace=False)
samplep_200 = np.random.choice(avail_purch_pids,size=200,replace=False)
print(len(midsell_200))

200


In [78]:
midsell_15good10_20

array(['8714784', '402401', '8487359', '4550250', '7823131', '8487340',
       '8823292', '7894949', '804202', '8404021', '8010978', '5759499',
       '8523562', '201797', '7607197', '7877929', '6304699', '9185188',
       '9030449', '10036752'], dtype=object)

In [75]:
avail_attr_purch.query('product_id in @midsell_10good15_20').head()

,product_id,prod_name,home_category_id,gmm_id,div_id,department_id,brand,prodtype,view_30_day_count,purch_30_day_count
2558692,367726,MAC STUDIO SCULPT SPF 15 FOUNDATION,30077,4,41,340,MAC,MAKEUP,3197,172
1972142,1505360,BALI ONE SMOOTH U ULTRALIGHT HIPSTER UNDERWEAR...,225,1,22,246,BALI,PANTY,6275,171
375760,9013598,FIRST IMPRESSIONS TODDLER GIRLS COTTON BIRD-PR...,63014,5,21,410,FIRST IMPRESSIONS,T-SHIRT,4656,171
3114,10133895,CIRCUS BY SAM EDELMAN CANYON MEDALLION FLAT SA...,17570,1,16,382,CIRCUS BY SAM EDELMAN,SHOE,14094,169
485450,8637964,"I.N.C. TIE-FRONT PEASANT TOP, CREATED FOR MACY'S",255,2,77,899,INC INTERNATIONAL CONCEPTS,TOP,7556,169


In [73]:
pid_list = midsell_10good15_20#worstsell_20
print(pid_list)
display_html(uu.html_recom_panel(pid_list, W=120),raw=True)

['6308269' '5759646' '2783187' '4849260' '1505360' '8930184' '10010758'
 '9013598' '8186135' '367726' '7980378' '7723851' '5132057' '10133895'
 '5837598' '6386570' '8301947' '8637964' '8757268' '9026560']


$39.98 (6308269) 
 XOXO Junio... 
 
 $55.0 (5759646) 
 Nike Men's... 
 
 $13.0 (2783187) 
 LANCME D... 
 
 $23.73 (4849260) 
 I.N.C. Men... 
 
 $8.4 (1505360) 
 BALI ONE S... 
 
 $59.5 (8930184) 
 Tommy Hilf... 
 
 $7.93 (10010758) 
 Champion L... 
 
 $15.0 (9013598) 
 First Impr... 
 
 $16.99 (8186135) 
 Jenni Stit... 
 
 $33.0 (367726) 
 MAC STUDIO... 
 
 $89.0 (7980378) 
 3/4 SLEEVE... 
 
 $48.0 (7723851) 
 Bar III Ir... 
 
 $99.99 (5132057) 
 HAMILTON B... 
 
 $? (10133895) 
 ?... 
 
 $250.0 (5837598) 
 Hotel Coll... 
 
 $49.99 (6386570) 
 Ayesha Cur... 
 
 $100.0 (8301947) 
 Juicy Cout... 
 
 $69.5 (8637964) 
 I.N.C. Tie... 
 
 $39.99 (8757268) 
 Charter Cl... 
 
 $56.0 (9026560) 
 Raisins Cu...